In [0]:
# PROJECT_PATH = '/home/dobos/project/pfs_targeting/python:' +\
#     '/home/dobos/project/pfsspec-all:' +\
#     '/home/dobos/project/pfs_cmdfit/python:' +\
#     '/home/dobos/project/pfs_isochrones/python:' +\
#     '/home/dobos/project/ics_cobraOps/python:' +\
#     '/home/dobos/project/ics_cobraCharmer/python:' +\
#     '/home/dobos/project/ets_fiberalloc:' +\
#     '/home/dobos/project/pfs_utils/python:' +\
#     '../pysynphot:' +\
#     '../SciScript-Python/py3'

ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc_nb_bosz'

# Umi
OBS_PATH = '/datascope/subaru/data/catalogs/dSph/ursaminor_tpall3e_g24.cat'
MEMBERS_PATH = '/datascope/subaru/data/cmdfit/dSph/spectro_members_umi.csv'
SIM_PATH = '/datascope/subaru/data/cmdfit/run/umi/sim/mix_bin_200k_hsc_007/sample.h5'


In [0]:
import os, sys
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle

# Plot style

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
# if 'debug' not in globals():
#     import debugpy
#     debugpy.listen(('0.0.0.0', 5698))
#     debug = True

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Configurations

In [0]:
configs = {}

for nb in [True, False]:
    for targeting in ['brightest', 'faintest', 'maxp']:
        c = {
            'nb': nb,
            'targeting': targeting
        }
        configs[targeting + ('' if not nb else '_nb')] = c

configs

# Photometry

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.common.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.common.photometry import Photometry, Magnitude, Color

In [0]:
hsc = SubaruHSC.photometry()
cmd = CMD([
        ColorAxis(
            Color([hsc.magnitudes['g'], hsc.magnitudes['i']]),
            limits=(-1, 4)),
        MagnitudeAxis(
            hsc.magnitudes['g'],
            limits=(15.5, 24.5)
        )
    ])
ccd = CCD([
    cmd.axes[0],
    ColorAxis(
        Color([hsc.magnitudes['g'], hsc.magnitudes['nb515']]),
        limits=(-0.5, 0.5)
    )
])

# Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

# Load observations

In [0]:
#obs = TextObservationReader().read_hsc(OBS_PATH)
obs = SubaruHSC.text_observation_reader().read(OBS_PATH)
obs.data.head()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

obs.plot(axs[0], cmd)
obs.plot(axs[1], ccd)

f.tight_layout()

In [0]:
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.common.diagram import FOV

In [0]:
ra, dec = obs.get_coords()
print(ra.mean(), dec.mean())

In [0]:
# Umi
pointing = Pointing(Angle('15h 09m 08.5s').degree, Angle('+67d 13m 21s').degree)
# Fornax
# pointing = Pointing(Angle('02h 39m 59s').degree, Angle('-34d 26m 57s').degree)
print(pointing.ra, pointing.dec)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=120, subplot_kw=dict(projection=wcs.wcs))
obs.plot(ax, fov)

# Load simulation

In [0]:
from pfs.ga.targeting.io import Hdf5SimulationReader

In [0]:
r = Hdf5SimulationReader()
cm = {}
for prefix in ['', 'obs_', 'err_', 'flux_', 'obs_flux_', 'err_flux_', 'counts_', 'obs_counts_', 'err_counts_']:
    cm[prefix + 'hsc_g2'] = prefix + 'hsc_g'
    cm[prefix + 'hsc_i2'] = prefix + 'hsc_i'
r.column_mapping = cm
r.append_photometry(SubaruHSC.photometry())
sim = r.read(SIM_PATH)

In [0]:
for k in sim.data.keys():
    print(k, sim.data[k].shape)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

sim.plot(axs[0], cmd, s=np.s_[::100], size=0.05)
sim.plot(axs[1], ccd, s=np.s_[::100], size=0.05)

f.tight_layout()

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

sim.plot(axs[0], cmd, apply_categories=True, s=np.s_[::20], size=0.05)
sim.plot(axs[1], ccd, apply_categories=True, s=np.s_[::20], size=0.05)

f.tight_layout()

# Define selections

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling, MagnitudeSelection, ColorSelection, LinearSelection

In [0]:
# Ursa Minor selection
if True:
    def get_selection_mask(catalog, nb=True, blue=False, probcut=None, observed=None, bright=16, faint=23.5):
        # Broadband colors
        mask = ColorSelection(ccd.axes[0], 0.12, 2.0).apply(catalog, observed=observed)

        # Narrow band
        if nb:
            mask &= (
                ColorSelection(ccd.axes[0], 0.12, 0.5).apply(catalog, observed=observed)

                | ColorSelection(ccd.axes[1], 0.1, None).apply(catalog, observed=observed)
                & ColorSelection(ccd.axes[0], None, 1.65).apply(catalog, observed=observed)
                
                | LinearSelection(ccd.axes, [-0.25, 1.0], -0.15, None).apply(catalog, observed=observed)
            )

        # Probability-based cut (map) - nonzero membership probability
        if probcut is not None:
            mask &= probcut.apply(catalog, observed=observed)

        # Allow blue
        if blue:
            mask |= (
                ColorSelection(ccd.axes[0], None, 0.12).apply(catalog, observed=observed)
            )

        # Always impose faint and bright magnitude cuts
        mask &= MagnitudeSelection(cmd.axes[1], bright, faint).apply(catalog, observed=observed)

        return mask

In [0]:
f, ax = plt.subplots(1, 1)

mask = get_selection_mask(obs, observed=True, nb=True, blue=True)
obs.plot(ax, cmd, observed=True, mask=mask)

# Isochrones

In [0]:
from pfs.ga.targeting import Isochrone

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3, 2), dpi=240)

hist, bins = np.histogram(sim.data['Z'][:, -2], density=True, bins=100)
ax.plot(0.5 * (bins[1:] + bins[:-1]), hist)
ax.set_xlabel('[Fe/H]')

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3, 2), dpi=240)

hist, bins = np.histogram(sim.data['t'][:, -2], density=True, bins=100)
#ax.plot(10**(0.5 * (bins[1:] + bins[:-1]) - 9), hist)
#ax.set_xlabel('age [Gyr]')
ax.plot(0.5 * (bins[1:] + bins[:-1]) - 9, hist)
ax.set_xlabel('log age')

In [0]:
iso_blue = Isochrone()
iso_blue.from_isogrid(hsc, iso, Fe_H=-2.5, log_t=10.135, DM=19.2)

iso_red = Isochrone()
#iso_red.from_isogrid(hsc, iso, Fe_H=-1.2, log_t=10.11, DM=19.2)
iso_red.from_isogrid(hsc, iso, Fe_H=-1.6, log_t=10.11, DM=19.2)

In [0]:
iso_blue.get_diagram_values(cmd.axes)

# Whitepaper plot

In [0]:
# Number of points within rectangular selection
mask = get_selection_mask(obs, nb=False, probcut=None, observed=True)
print('obs:', mask.shape, mask.sum())

mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)
mask = sim.apply_categories(mask)
print('sim:', mask.shape, mask.sum(axis=0))

In [0]:
w1 = np.array([2.05946072e-04, 8.57202980e-05, 3.65160318e-02, 1.48482228e-02,
 5.69068666e-04, 2.49689236e-04, 5.41036362e-02, 2.27939918e-02,
 1.33515545e-01, 5.77029378e-02, 4.76497084e-01, 2.02912126e-01])

w2 = np.array([2.12000e-04, 7.60000e-05, 3.66160e-02, 1.47720e-02, 5.36000e-04, 2.88000e-04,
 5.34720e-02, 2.30760e-02, 1.34424e-01, 5.76160e-02, 4.77500e-01, 2.01412e-01])

In [0]:
# Population weights without NB cuts
print('w1', w1, w1.sum())

# New categories
g1 = np.random.choice(np.arange(w1.size, dtype=int), sim.data['g'].size, p=w1)
print('g1', g1.shape)

# Verify new categories
w2 = np.bincount(g1) / g1.shape
print('w2', w2, w2.sum(), np.sum(w2[:-2]), np.sum(w2[-2:]))

# Population weights after cuts
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)
sim.apply_categories(mask, g=g1)
w3 = np.bincount(g1[mask[:,0]]) / mask.sum()
w3 /= w3.sum()
print('w3', w3, np.sum(w3), np.sum(w3[:-2]), np.sum(w3[-2:]))

# Population weights for the ghost plot
w = np.bincount(sim.data['g']) / sim.data['g'].shape
print('w', w, np.sum(w[:-2]), np.sum(w[-2:]))

# New weights, boost thick disk and halo
w4 = np.r_[w[:-2] / 0.4 * 0.5, w[-2:] / 0.6 * 0.5]
w4[2:4] *= 100
# w4[0:6] *= 3  # thin disk 1-3
# w4[4:6] *= 1.2  # thin disk 3
w4[6:8] *= 15  # thick disk
w4[8:10] *= 28   # halo
w4[10:12] *= 50  # dSph
w4 /= w4.sum()
print('w4', w4, np.sum(w4[:-2]), np.sum(w4[-2:]))

In [0]:
mask = get_selection_mask(sim, nb=False, probcut=None, observed=True)

pmap_nonb = ProbabilityMap(cmd.axes)
pmap_nonb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w1, observed=True, mask=mask)
pmap_nonb.maximum_filter()

pmap_samp = ProbabilityMap(cmd.axes)
pmap_samp.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w4, observed=True, mask=mask)
pmap_samp.maximum_filter()

mask = get_selection_mask(sim, nb=True, probcut=None, observed=True)

pmap_nb = ProbabilityMap(cmd.axes)
pmap_nb.from_simulation(sim, bins=[100, 100], extents=[[0.12, 2.0], [16, 23.5]], 
    merge_list=[np.s_[:10], np.s_[10:]], population_weights=w3, observed=True, mask=mask)
pmap_nb.maximum_filter()

In [0]:
psamp = ProbabilitySampling(pmap_samp, 0)
psamp_mask = psamp.apply(obs, observed=True)

In [0]:
def hex_to_rgb(hex):
    return tuple(int(hex[i + 1:i + 3], 16) / 255 for i in (0, 2, 4))

def chop_cmap_frac(cmap: LinearSegmentedColormap, frac: float) -> LinearSegmentedColormap:
    """Chops off the beginning `frac` fraction of a colormap."""
    cmap_as_array = cmap(np.arange(256))
    cmap_as_array = cmap_as_array[:int(frac * len(cmap_as_array))]
    return LinearSegmentedColormap.from_list(cmap.name + f"_frac{frac}", cmap_as_array)

def get_nice_cmap(from_black=False):
    if from_black:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, hex_to_rgb('#595959')),
            (0.33, hex_to_rgb('#5F9ED1')),
            (0.66, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad((1.0, 1.0, 1.0))
    else:
        cmap = LinearSegmentedColormap.from_list('mycmap',
            [(0.00, (1.0, 1.0, 1.0)),
            (0.15, hex_to_rgb('#ABABAB')),
            (0.25, hex_to_rgb('#595959')),
            (0.50, hex_to_rgb('#5F9ED1')),
            (0.75, hex_to_rgb('#006BA4')),
            (1.00, hex_to_rgb('#FF800E'))])

        cmap.set_bad(cmap(0))

    return cmap

#cmap = mpl.cm.get_cmap('terrain_r')

# cmap = LinearSegmentedColormap.from_list('mycmap',
#         [(0.00, (1.0, 1.0, 1.0)),
#          #(0.15, (0.75, 0.75, 0.75)),
#          (0.25, (1.0, 1.0, 0.6)),
#          (0.50, (0.0, 0.8, 0.4)),
#          (0.75, (0.0, 0.6, 1.0)),
#          (1.00, (1.0, 0.0, 0.0))])

#006BA4, #FF800E, #ABABAB, #595959,
#5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF

# cmap = mpl.cm.get_cmap('cubehelix_r')
# cmap = chop_cmap_frac(cmap, 0.5)

cmap = get_nice_cmap()

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap())

In [0]:
plt.imshow(np.stack(20 * [np.linspace(0, 1, 1000)], axis=0), cmap=get_nice_cmap(from_black=True))

In [0]:
gs = GridSpec(ncols=6, nrows=1, width_ratios=[5, 5, 5, 5, 5, 0.3])
f = plt.figure(constrained_layout=False, figsize=(7.2, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(5) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

obs.plot(axs[0], cmd, c='grey', observed=True, rasterized=True)
#cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
iso_blue.plot(axs[0], cmd, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
iso_red.plot(axs[0], cmd, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

sim.plot(axs[1], cmd, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

obs.plot(axs[2], cmd, mask=psamp_mask, c='grey', rasterized=True)
l0 = pmap_nonb.plot(axs[3], cmd, 1, cmap=cmap, rasterized=True)
l1 = pmap_nb.plot(axs[4], cmd, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min', cmap=cmap)
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.55, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')

In [0]:
probcmap = plt.get_cmap('Greys')

gs = GridSpec(ncols=7, nrows=1, width_ratios=[5, 5, 5, 5, 0.3, 0.6, 0.3])
f = plt.figure(constrained_layout=False, figsize=(4.5, 1.8), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(4) ])
axc = f.add_subplot(gs[0, -2])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

obs.plot(axs[0], cmd, c='grey', observed=True, rasterized=True)
# cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

sim.plot(axs[1], cmd, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

obs.plot(axs[2], cmd, mask=psamp_mask, c='grey', rasterized=True)
l0 = pmap_nonb.plot(axs[3], cmd, 1, cmap=probcmap, rasterized=True)
# l1 = cmd.plot_probability_map(axs[4], pmap_nb, 1, cmap=probcmap, rasterized=True)

for l in (l0,):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min', cmap=probcmap)
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=0.5)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.3, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'$\mathrm{mag}~g - \mathrm{mag}~i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.55, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'$\mathrm{mag}~g$')

axs[0].set_title('observation')
axs[1].set_title('simulation')
axs[2].set_title('"ghost plot"')
axs[3].set_title('probability')

f.subplots_adjust(wspace=0)
# f.tight_layout()

f.savefig('obs_sim_prob_poster.pdf')
f.savefig('obs_sim_prob_poster.eps')

In [0]:
gs = GridSpec(ncols=3, nrows=1, width_ratios=[10, 10, 0.5])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])
axc = f.add_subplot(gs[0, -1])

#f, axs = plt.subplots(1, 4, figsize=(7.2, 2.5), dpi=300)

# cmd.plot_observation(axs[0], obs, c='grey', observed=True, rasterized=True)
# #cmd.plot_observation(axs[0], obs, mask=confirmed_mask, c='r', size=1)
# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

# cmd.plot_simulation(axs[1], sim, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)
# #cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-1.0, -1.0], c='blue')
# #cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[1.0, 1.0], c='red')

# cmd.plot_observation(axs[0], obs, mask=psamp_mask, c='grey', rasterized=True)

l0 = pmap_nonb.plot(axs[0], cmd, 1, cmap=cmap, rasterized=True)
l1 = pmap_nb.plot(axs[1], cmd, 1, cmap=cmap, rasterized=True)

for l in (l0, l1):
    #l0.set_clim(min([l.get_clim()[0] for l in (l0, l1)]), max([l.get_clim()[1] for l in (l0, l1)]))
    l.set_clim(-3.1, 0)

#l1.set_clim(*l0.get_clim())

print(l0.get_clim())

c = f.colorbar(l0, cax=axc, orientation='vertical', extend='min')
c.set_label('$\log_{10} \,\, p_\mathrm{member}$')

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'abcde'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel(None)
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()
    
for ax in axs.flatten()[1:]:
    ax.set_yticklabels([])
    
axs[0].set_ylabel(r'HSC $g$')

for t in axc.get_yticklabels():
    t.set_horizontalalignment('right')   
    t.set_verticalalignment('center')   
    t.set_x(4.2)
    #t.set_y(-1)

f.tight_layout()

f.savefig('umi_hsc_pmap.pdf')

In [0]:
get_selection_mask(obs)

In [0]:
gs = GridSpec(ncols=2, nrows=1, width_ratios=[10, 10])
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.1), dpi=240)

axs = np.array([ f.add_subplot(gs[0, i]) for i in range(2) ])

obs.plot(axs[0], cmd, mask=psamp_mask, c='grey', rasterized=True)

psamp_outline_mask = get_selection_mask(obs, nb=True, probcut=None, observed=True)
obs.plot(axs[1], cmd, mask=~psamp_mask & psamp_outline_mask, c='grey', rasterized=True)

for ax in axs:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax, pl in zip(axs.flatten(), 'ab'):
    ax.text(-0.4, 16.2, f'{pl})', fontsize=8)
    ax.set_xlabel(r'HSC $g - i$')
    ax.set_ylabel('HSC $g$')
    ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    ax.set_ylim(15.5, 24.5)
    ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)
    ax.ticklabel_format()

for ax in axs.flatten()[1:]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

f.tight_layout()

f.savefig('umi_hsc_ghost.pdf')

In [0]:
f, axs = plt.subplots(2, 2, figsize=(3.4, 4), dpi=240)

mask = get_selection_mask(obs, nb=True)

obs.plot(axs[0, 0], cmd, c='grey', observed=True, rasterized=True)
iso_blue.plot(axs[0, 0], cmd, observed=True, error_sigma=[-2.0, -2.0], c=hex_to_rgb('#0000FF'), lw=1)
iso_red.plot(axs[0, 0], cmd, observed=True, error_sigma=[2.0, 2.0], c=hex_to_rgb('#FF800E'), lw=1)

sim.plot(axs[0, 1], cmd, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

# mask = get_selection_mask(obs, nb=True)

mask = get_selection_mask(obs, nb=True, faint=22)

obs.plot(axs[1, 0], ccd, c='grey', observed=True, rasterized=True)
# obs.plot(axs[1, 0], ccd, c='red', mask=mask, observed=True, rasterized=True)

sim.plot(axs[1, 1], ccd, s=np.s_[::20], observed=True, apply_categories=True, g=g1, c='grey', rasterized=True)

for ax, pl in zip(axs.flatten(), 'abcd'):
    ax.text(0.1, 0.9, f'{pl})', fontsize=8, transform=ax.transAxes)
    ax.set_xlabel(r'HSC $g - i$')
    #ax.yaxis.set_ticks(np.arange(16, 24.5, 1.0))
    #ax.xaxis.set_ticks([0, 0.5, 1, 1.5])
    ax.set_xlim(-0.75, 2.4)
    #ax.set_ylim(15.5, 24.5)
    #ax.invert_yaxis()
    #ax.grid()
    
    ax.tick_params(direction="in", top=True, right=True)

for ax in axs[0, :]:
    #ax.plot([0, 1.3, 1.3, 0, 0], [17, 17, 23.5, 23.5, 17], '--k', lw=0.5)
    ax.plot([0.1, 2.0, 2.0, 0.1, 0.1], [16.1, 16.1, 23.5, 23.5, 16.1], '--k', lw=1)

for ax in axs[1, :]:
    ax.plot([0.12, 2.0, 2.0,
         1, 0.5, 0.5, 0.12, 0.12], 
        
        [0.5, 0.5, 0.1,
         0.1, -0.025, -0.5, -0.5, 0.5], 'k--', lw=1)

axs[0, 0].set_ylabel(r'HSC $g$')
axs[1, 0].set_ylabel(r'HSC $g - $NB$515$')

for ax in axs[:, 1]:
    ax.set_ylabel(None)
    ax.set_yticklabels([])

for ax in axs[0, :]:
    ax.set_xlabel(None)
    ax.set_xticklabels([])

f.tight_layout()

f.savefig('umi_hsc_cmd.pdf')

# Plot selection

In [0]:
def plot_selection_mask(catalog, mask, s=np.s_[:]):
    f, axs = plt.subplots(2, 2, figsize=(6, 6), dpi=120)

    catalog.plot(axs[0, 0], cmd, mask=~mask, s=s, c='k')
    catalog.plot(axs[0, 0], cmd, mask=mask, s=s, c='r')
    catalog.plot(axs[0, 1], cmd, mask=mask, s=s, c='k')

    catalog.plot(axs[1, 0], ccd, mask=~mask, s=s, c='k')
    catalog.plot(axs[1, 0], ccd, mask=mask, s=s, c='r')
    catalog.plot(axs[1, 1], ccd, mask=mask, s=s, c='k')

    f.tight_layout()

In [0]:
mask = get_selection_mask(obs, nb=False)
plot_selection_mask(obs, mask)

# Fit the radial profile

Here we fit to all stars **within the coarse color cuts** to have a better fit of the shape parameters. Later the foreground and the central density will be fitted again while keeping the morphological parameters constant. The initial foreground/member fraction will be determined from the radial profile fit.

In [0]:
from pfs.ga.targeting.projection import Pointing, Whitening
from pfs.ga.targeting.radialprofile import *

In [0]:
from astropy.time import Time

In [0]:
# Cobra_charmer requires a valid logging handler
import logging
logging.getLogger().addHandler(logging.NullHandler())

In [0]:
# Define pointings
from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
field = DSPH_FIELDS['ursaminor']

config = field.get_netflow_config()
center = field.get_center()
pointings = field.get_pointings(SubaruPFI)
obs_time = Time('2026-01-15T10:00:00')        # midnight in Hawaii
wfc = SubaruWFC(center)
# pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)
wcs = WcsProjection(center, proj='TAN')
fov = FOV(projection=wcs)

In [0]:
center

In [0]:
# Only the coarse color cuts
mask = get_selection_mask(obs, nb=True, probcut=None)

In [0]:
plot_selection_mask(obs, mask)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120,
    subplot_kw={'projection': fov.projection.wcs})

obs.plot(axs[0], fov)
obs.plot(axs[0], fov, mask=mask, color='r')

obs.plot(axs[1], fov, mask=mask)


In [0]:
w = Whitening(projection=wcs)

# Ursa Minor
w_mask = w.create(obs, iterations=25, s_cut=2.2)

w.M, w.reverse(w.M), field.pos

In [0]:
(x, y), _ = w.projection.world_to_pixel(obs.get_coords())
wxy = w.apply(obs)
x.shape, y.shape, wxy.shape, w.M

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(x, y, 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(x[w_mask], y[w_mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(x[mask], y[mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

In [0]:
wrr = np.array([[0, 0], [0, 5], [5, 0]])

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(wxy[..., 0], wxy[..., 1], 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(wxy[..., 0][w_mask], wxy[..., 1][w_mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(wxy[..., 0][mask], wxy[..., 1][mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

axs[1].plot(wrr[..., 0], wrr[..., 1], 'xr')

In [0]:
xy = np.matmul(np.linalg.inv(w.W), wxy.T).T + w.M
rr = np.matmul(np.linalg.inv(w.W), wrr.T).T + w.M

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

axs[0].plot(xy[..., 0], xy[..., 1], 'sk', markeredgecolor='None', ms=0.5)
axs[0].plot(xy[..., 0][mask], xy[..., 1][mask], 'sr', markeredgecolor='None', ms=0.5)
axs[0].set_aspect(1)

axs[1].plot(xy[..., 0][mask], xy[..., 1][mask], 'sk', markeredgecolor='None', ms=0.5)
axs[1].set_aspect(1)

axs[1].plot(rr[..., 0], rr[..., 1], 'xr')

In [0]:
# Fit profile to all stars within the color cut

king = King(transformation=w, R_max=6, bins=30)
king.histogram(obs, mask=mask)
[S_b, S_0, R_c, R_t], pcov = king.fit()

S_b, S_0, R_c, R_t

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=120)

R, log_S, log_S_sigma = king.get_log_S()

ax.errorbar(R, log_S, 3 * log_S_sigma)
ax.set_xlabel('deprojected R')
ax.set_ylabel(r'$\log \Sigma$')

R = np.linspace(0, 8, 200)
log_S = king.log_eval(R)
ax.plot(R, log_S)

In [0]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [0]:
# king.transformation.projection.pointing.posang

# king.transformation.Vh

np.arctan2(king.transformation.Vh[0, 1], king.transformation.Vh[0, 0])

In [0]:
ra0, dec0 = king.transformation.reverse(0, 0)
ra1, dec1 = king.transformation.reverse(0, 1)
print(ra, dec)

In [0]:
f = plt.figure(constrained_layout=False, figsize=(3.4, 2.), dpi=240)
ax = f.add_subplot(projection=wcs.wcs)

obs.plot(ax, fov, c='gray', size=0.02, rasterized=True)

ell = king.get_ellipse(R_t)
# fov.plot(ax, ell, native_frame='world', fmt='--r')

(S_b, S_0, R_c, R_t) = king.params
R = np.array([0, R_t, R_t, R_t, R_t])
phi = np.array([0, 0, np.pi / 2, np.pi, 3 / 2 * np.pi])

ra, dec = king.elliptic_to_world(R, phi, ctype='t')

# c = SkyCoord(ra=ra * u.degree, dec=dec * u.degree, frame='icrs')
# ax.plot(c.ra.degree, c.dec.degree, 'x', transform=ax.get_transform('world'))
ax.set_xlim(-2, 1)
ax.set_ylim(-2.5, 0.6)
ax.invert_xaxis()
ax.grid()

In [0]:
obs.get_coords()[0].min(), obs.get_coords()[0].max()

In [0]:
obs.get_coords()[0].mean(), obs.get_coords()[1].mean()

In [0]:
gs = GridSpec(ncols=2, nrows=1, width_ratios=[3, 2])
f = plt.figure(constrained_layout=False, figsize=(3.5, 2.1), dpi=240)

axs = 2 * [None]

axs[0] = f.add_subplot(gs[0], projection=fov.projection.wcs)

(S_b, S_0, R_c, R_t) = king.params
xy = w.apply(obs)
r = np.sqrt(xy[:, 0]**2 + xy[:, 1]**2)

# mask = get_selection_mask(obs, observed=True, nb=True, blue=True, probcut=probcut)
# mask &= r > R_t 

tidal_mask = r > R_t

obs.plot(axs[0], fov, mask=tidal_mask, c=hex_to_rgb('#FF800E'), size=0.02, rasterized=True)
obs.plot(axs[0], fov, mask=~tidal_mask, c='gray', size=0.02, rasterized=True)

ell = king.get_ellipse(R_t, res=36)
fov.plot(axs[0], ell, fmt='--', native_frame='world', color=hex_to_rgb('#FF800E'), ls='--')

#ell = king.get_ellipse(R_c, res=36)
#fov.plot(axs[0], ell, fmt='-', color=hex_to_rgb('#FF800E'))

# for pp in [(228.2, 67.5, 0),
#            (226.3, 67.5, 0),
#            (226.0, 66.9, 0),
#            (228.1, 66.955, 40)]:
    
for pp in pointings:
    pp.obs_time = obs_time

    if pp.stage <= 1:
        # wfc = SubaruWFC(Pointing(227.3, 67.25))
        wfc = SubaruWFC(pp)

        pfi = SubaruPFI(wfc, instrument_options=config.instrument_options)
        pfi.plot_focal_plane(axs[0], fov, corners=True, c=hex_to_rgb('#006BA4'), ls='-')

axs[0].text(0.05, 0.9, 'a)', fontsize=8, transform=axs[0].transAxes)
axs[0].set_xlabel(r'right ascension')
axs[0].set_ylabel(r'declination')
#axs[0].yaxis.set_ticks(np.arange(16, 24.5, 1.0))
#axs[0].xaxis.set_ticks([0, 0.5, 1, 1.5])
#axs[0].set_xlim(-0.55, 2.4)
#axs[0].set_ylim(15.5, 24.5)

axs[0].set_xlim(-2.3, 0.5)
axs[0].set_ylim(-2.3, 0.5)

axs[0].invert_xaxis()
#axs[0].grid()

axs[0].tick_params(direction="in") #, top=True, right=True)
axs[0].grid()

####

axs[1] = f.add_subplot(gs[1])

obs.plot(axs[1], cmd, mask=tidal_mask, color='grey', size=0.02, rasterized=True)
#cmd.plot_catalog(axs[1], obs, mask=mask, color='red')

#cmd.plot_isochrone(axs[1], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
#cmd.plot_isochrone(axs[1], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)

####

# axs[2] = f.add_subplot(gs[2])

# cmd.plot_catalog(axs[2], obs, mask=mask, color='grey')

####

axs[1].set_aspect('auto')

axs[1].text(0.075, 0.9, 'b)', fontsize=8, transform=axs[1].transAxes)
axs[1].set_xlabel(r'HSC $g - i$')
axs[1].set_ylabel(r'HSC $g$')

axs[1].yaxis.tick_right()
axs[1].yaxis.set_label_position("right")
axs[1].tick_params(direction="in", left=True, bottom=True, top=True, right=True)

f.tight_layout()

p0 = axs[0].get_position()
for ax in axs[1:]:
    p1 = ax.get_position()
    p1 = [p1.x0, p0.y0, p1.width, p0.height]
    ax.set_position(p1)

f.savefig('umi_hsc_tidal.pdf')